In [117]:
import pandas as pd

def convert_year_to_4_digits(date_str):
    """Convert 2-digit year to 4-digit year by adding '20' prefix"""
    if isinstance(date_str, str) and '/' in date_str:
        parts = date_str.split('/')
        if len(parts) == 3 and len(parts[2]) == 2:
            parts[2] = '20' + parts[2]
            return '/'.join(parts)
    return date_str

def convert_data_format(df: pd.DataFrame, format: str = '%d/%m/%Y') -> pd.DataFrame:
    df['Compra'] = df['Compra'].apply(lambda x:  convert_year_to_4_digits(x) )
    df['Compra'] = pd.to_datetime(df['Compra'], format=format, errors='coerce')
    df['Inserção'] = df['Inserção'].apply(lambda x:  convert_year_to_4_digits(x) )
    df['Inserção'] = pd.to_datetime(df['Inserção'], format=format, errors='coerce')
    return df
    
def clean_price(price_str):
    """Convert Brazilian price format to numeric value"""
    if isinstance(price_str, str):
        # Remove 'R$' and whitespace, replace comma with dot
        cleaned = price_str.replace('R$', '').strip().replace(',', '.')
        try:
            return float(cleaned)
        except ValueError:
            return None
    return price_str

# Create a function to swap columns 
def swap_columns(df, col1, col2):
    """Swap two columns in a dataframe using values from a temporary dataframe"""
    temp_df = df.copy()

    df[col1] = temp_df[col2]
    df[col2] = temp_df[col1]


year = 2024
# read data
df_2020_original = pd.read_csv(f"../data/sus/2020.csv", sep=';', encoding='ISO-8859-1', quotechar='"')
df_2021_original = pd.read_csv(f"../data/sus/2021.csv", sep=';', encoding='ISO-8859-1', quotechar='"')
df_2022_original = pd.read_csv(f"../data/sus/2022.csv", sep=';', encoding='ISO-8859-1', quotechar='"')
df_2023_original = pd.read_csv(f"../data/sus/2023.csv", sep=';', encoding='utf-8', quotechar='"')
df_2024_original = pd.read_csv(f"../data/sus/2024.csv", sep=';', encoding='utf-8', quotechar='"')

#get copy
df_2020 = df_2020_original.copy()
df_2021 = df_2021_original.copy()
df_2022 = df_2022_original.copy()
df_2023 = df_2023_original.copy()
df_2024 = df_2024_original.copy()

# transform data
df_2020 = convert_data_format(df_2020)
df_2021 = convert_data_format(df_2021)
df_2022 = convert_data_format(df_2022)
df_2023 = convert_data_format(df_2023, format = '%m/%d/%Y')

# transform 2024 data
df_2024['Compra'] = pd.to_datetime('1899-12-30') + pd.to_timedelta(df_2024['Compra'], unit='D')
df_2024['Inserção'] = pd.to_datetime('1899-12-30') + pd.to_timedelta(df_2024['Inserção'], unit='D')


# Define columns for each year
columns_3_first_years = ['ano', 'codigo_br', 'descricao_catmat', 'unidade_de_fornecimento',
    'generico', 'anvisa', 'compra', 'modalidade_da_compra', 'insercao',
    'tipo_compra', 'fabricante', 'cnpj_fabricante', 'fornecedor',
    'cnpj_fornecedor', 'nome_instituicao', 'cnpj_instituicao',
    'municipio_instituicao', 'uf', 'qtd_itens_comprados', 'preco_unitario']


columns_last_2_years = ['nome_instituicao', 'cnpj_instituicao', 'municipio_instituicao', 'uf',
    'compra', 'insercao', 'modalidade_da_compra', 'codigo_br',
    'descricao_catmat', 'unidade_fornecimento', 'generico', 'anvisa',
    'cnpj_fornecedor', 'fornecedor', 'cnpj_fabricante', 'fabricante',
    'qtd_itens_comprados', 'preco_unitario', 'preco_total']


df_2020.columns = columns_3_first_years
df_2021.columns = columns_3_first_years
df_2022.columns = columns_3_first_years
df_2023.columns = columns_last_2_years
df_2024.columns = columns_last_2_years 


# Apply swaps to all three dataframes
for df in [df_2020]:
    # Swap fabricante and cnpj_fabricante
    swap_columns(df , 'fabricante', 'cnpj_fabricante')
    
    # Swap fornecedor and cnpj_fornecedor
    swap_columns(df, 'fornecedor', 'cnpj_fornecedor')
    
    # Swap nome_instituicao and cnpj_instituicao
    swap_columns(df, 'nome_instituicao', 'cnpj_instituicao')


# Apply the conversion to the preco_unitario column
df_2020['preco_unitario'] = df_2020['preco_unitario'].apply(clean_price)
df_2021['preco_unitario'] = df_2021['preco_unitario'].apply(clean_price)
df_2022['preco_unitario'] = df_2022['preco_unitario'].apply(clean_price)
df_2023['preco_unitario'] = df_2023['preco_unitario'].apply(clean_price)
df_2024['preco_unitario'] = df_2024['preco_unitario'].apply(clean_price)

# Convert 'qtd_itens_comprados' to Int64 type

df_2020['qtd_itens_comprados'] = df_2020['qtd_itens_comprados'].astype(str).str.replace('.', '').astype('Int64')
df_2021['qtd_itens_comprados'] = df_2021['qtd_itens_comprados'].astype(str).str.replace('.', '').astype('Int64')
df_2022['qtd_itens_comprados'] = df_2022['qtd_itens_comprados'].astype(str).str.replace('.', '').astype('Int64')
df_2023['qtd_itens_comprados'] = df_2023['qtd_itens_comprados'].astype(str).str.replace('.', '').astype('Int64')
df_2024['qtd_itens_comprados'] = df_2024['qtd_itens_comprados'].astype(str).str.replace('.', '').astype('Int64')

# Standardize column names and order for concatenation
standard_columns = [
    'nome_instituicao', 'cnpj_instituicao', 'municipio_instituicao', 'uf',
    'compra', 'insercao', 'modalidade_da_compra', 'codigo_br',
    'descricao_catmat', 'unidade_fornecimento', 'generico', 'anvisa',
    'cnpj_fornecedor', 'fornecedor', 'cnpj_fabricante', 'fabricante',
    'qtd_itens_comprados', 'preco_unitario', 'preco_total'
]

# Rename 'unidade_de_fornecimento' to 'unidade_fornecimento' in first 3 years to match standard
df_2020 = df_2020.rename(columns={'unidade_de_fornecimento': 'unidade_fornecimento'})
df_2021 = df_2021.rename(columns={'unidade_de_fornecimento': 'unidade_fornecimento'})
df_2022 = df_2022.rename(columns={'unidade_de_fornecimento': 'unidade_fornecimento'})

# Add missing 'preco_total' column to first 3 years (calculate from qty * unit_price)
df_2020['preco_total'] = df_2020['qtd_itens_comprados'] * df_2020['preco_unitario']
df_2021['preco_total'] = df_2021['qtd_itens_comprados'] * df_2021['preco_unitario']
df_2022['preco_total'] = df_2022['qtd_itens_comprados'] * df_2022['preco_unitario']

df_2023['tipo_compra'] = ''
df_2024['tipo_compra'] = ''


# Remove 'ano' column from first 3 years and reorder all dataframes
df_2020 = df_2020.drop(['ano'], axis=1)[standard_columns]
df_2021 = df_2021.drop(['ano'], axis=1)[standard_columns]
df_2022 = df_2022.drop(['ano'], axis=1)[standard_columns]
df_2023 = df_2023[standard_columns]
df_2024 = df_2024[standard_columns]




In [118]:
df = pd.concat([df_2020, df_2021, df_2022, df_2023, df_2024], ignore_index=True)

df['modalidade_da_compra'] = df['modalidade_da_compra'].str.strip()

In [119]:
df['generico'] = df['generico'].apply(lambda x: True if x in ('Sim', 'S') else (False if x in ('Não', 'N') else None))

In [120]:

df_ = convert_data_format(df_2023_original)  # Ensure the original data is also converted
df_.isna().sum() / df_.shape[0] * 100

Nome Instituição          0.010660
CNPJ Instituição          0.000000
Município Instituição     0.000000
UF                        0.000000
Compra                   60.988220
Inserção                 68.701029
Modalidade da Compra      0.000000
Código BR                 0.000000
Descrição CATMAT          0.000000
Unidade Fornecimento      0.154576
Genérico                  0.000000
ANVISA                   55.063696
CNPJ Fornecedor           0.000000
Fornecedor                0.000000
CNPJ Fabricante           0.000000
Fabricante                0.000000
Qtd Itens Comprados       0.000000
Preço Unitário            0.000000
Preço Total               0.000000
dtype: float64

In [121]:
# Now save to parquet
df.to_csv('../data/sus/sus_data.csv', index=False)

## Municípios

In [122]:
municipios = pd.read_csv('../data/sus/municipios.csv')
estados = pd.read_csv('../data/sus/estados.csv')

# Normalize municipality names to uppercase and remove accents
municipios['nome'] = municipios['nome'].str.upper().str.normalize('NFD').str.encode('ascii', errors='ignore').str.decode('ascii')

municipios = pd.merge(municipios, estados[['codigo_uf', 'uf']], on='codigo_uf', how='left', suffixes=('', '_estado'))

municipios = municipios[['nome', 'latitude', 'longitude', 'uf']]
df['municipio_instituicao'] = df['municipio_instituicao']\
    .str.replace('MUNICIPIO DE ', '', regex=False)\
    .str.replace('FUNDO MUNICIPAL DE ', '', regex=False)\
    .str.rstrip()

df_geo = pd.merge(df, municipios, left_on=['municipio_instituicao','uf'], right_on=['nome', 'uf'], how='left')

df_geo.to_csv('../data/sus/sus_data_geo.csv', index=False)

In [115]:
df_2023_original

,Nome Instituição,CNPJ Instituição,Município Instituição,UF,Compra,Inserção,Modalidade da Compra,Código BR,Descrição CATMAT,Unidade Fornecimento,Genérico,ANVISA,CNPJ Fornecedor,Fornecedor,CNPJ Fabricante,Fabricante,Qtd Itens Comprados,Preço Unitário,Preço Total
0,FUNDO MUNICIPAL DE SAUDE DE MARABA,18.478.187/0001-07,MARABA,PA,01/01/23,03/15/24,Pregão,348282,"REAGENTE, COMPONENTES 3:COM ANTICORPOS MONOCLO...",NaN,N,NaN,05.048.534/0001-01,NORTEMED DISTRIBUIDORA DE PRODUTOS MEDICOS LTDA,50.657.402/0001-31,EBRAM PRODUTOS LABORATORIAIS LTDA,220,"35,36","7779,2"
1,FUNDO MUNICIPAL DE SAUDE DE MARABA,18.478.187/0001-07,MARABA,PA,01/01/23,03/15/24,Pregão,395610,"REAGENTE ANALÍTICO 4, TIPO:ÁLCOOL-ÁCIDO, CONCE...",UNIDADE,N,NaN,41.733.464/0001-94,SHL - SAUDE HOSPITALAR E LABORATORIAL LTDA,00.562.583/0001-44,RENYLAB - QUIMICA E FARMACEUTICA LTDA,55,"48,49","2666,95"
2,FUNDO MUNICIPAL DE SAUDE DE MARABA,18.478.187/0001-07,MARABA,PA,01/01/23,03/15/24,Pregão,357684,"ÓLEO DE IMERSÃO, USO:PARA MICROSCOPIA, ASPECTO...",FRASCO,N,NaN,07.944.100/0001-15,PROC9 INDUSTRIA QUIMICA LTDA,07.944.100/0001-15,PROC9 INDUSTRIA QUIMICA LTDA,150,11,1650
3,FUNDO MUNICIPAL DE SAUDE DE MARABA,18.478.187/0001-07,MARABA,PA,01/01/23,03/15/24,Pregão,399068,"CORANTE, TIPO :CORANTE DE GIEMSA, ASPECTO FÍSI...",FRASCO,N,NaN,07.944.100/0001-15,PROC9 INDUSTRIA QUIMICA LTDA,07.944.100/0001-15,PROC9 INDUSTRIA QUIMICA LTDA,34,42,1428
4,FUNDO MUNICIPAL DE SAUDE DE MARABA,18.478.187/0001-07,MARABA,PA,01/01/23,03/15/24,Pregão,347476,"ÓXIDO DE MERCÚRIO II, ASPECTO FÍSICO:CRISTAIS ...",GRAMA,N,NaN,07.944.100/0001-15,PROC9 INDUSTRIA QUIMICA LTDA,07.944.100/0001-15,PROC9 INDUSTRIA QUIMICA LTDA,3,160,480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37517,NaN,06.205.888/0001-85,SOBRADINHO,RS,12/29/23,NaN,Registro de Preços,267503,"ÁCIDO FÓLICO, DOSAGEM:5 MG",COMPRIMIDO,N,1.134302e+12,06.294.126/0001-00,RS - PRODUTOS HOSPITALARES LTDA,19.570.720/0001-10,HIPOLABOR FARMACEUTICA LTDA,456000,"0,037",16872
37518,MUNICIPIO DE DOURADOQUARA,18.158.261/0001-08,DOURADOQUARA,MG,12/29/23,07/12/24,Registro de Preços,267651,"ENALAPRIL MALEATO, DOSAGEM:10 MG",COMPRIMIDO,N,NaN,03.945.035/0001-91,ACACIA COMERCIO DE MEDICAMENTOS LTDA,33.150.764/0001-12,BIOLAB FARMA GENERICOS LTDA,4800,"0,0275",132
37519,SECRETARIA DE ESTADO DE SAUDE - SES,42.498.717/0001-55,RIO DE JANEIRO,RJ,12/29/23,07/22/24,Pregão,478270,"LANADELUMABE, CONCENTRAÇÃO:150 MG/ML, FORMA FA...",FRASCO,N,1.063903e+12,10.586.940/0001-68,ONCOVIT DISTRIBUIDORA DE MEDICAMENTOS LTDA,60.397.775/0001-74,TAKEDA PHARMA LTDA.,32,"40144,86","1284635,52"
37520,MUNICIPIO DE SENADOR SA,07.598.642/0001-83,SENADOR SA,CE,12/29/23,06/21/24,Pregão,386131,"EQUIPO, TIPO DE EQUIPO:P/NUTRIÇÃO ENTERAL, MAT...",UNIDADE,N,NaN,21.116.490/0001-66,F B COMERCIO DE PRODUTOS E EQUIPAMENTOS LTDA,09.284.952/0001-59,KEDRION BRASIL DISTRIBUIDORA DE PRODUTOS HOSPI...,2000,"1,96",3920


## API

In [48]:
import requests

url = "https://apidadosabertos.saude.gov.br/economia-da-saude/bps?limit=100&offset=1000"

response =  requests.get(url)

In [49]:
response.json()

{'bps': [{'insercao': '45582',
   'municipio_da_instituicao': 'SAO JOSE DE CAIANA',
   'quantidade_de_itens_comprados': '1000',
   'preco_unitario': '2,84',
   'codigo_br': '268504',
   'preco_total': '2840',
   'unidade_de_fornecimento': 'AMPOLA',
   'fabricante': 'HIPOLABOR FARMACEUTICA LTDA',
   'cnpj_da_instituicao': '10.785.644/0001-96',
   'generico': 'N',
   'fornecedor': 'ALLFAMED COMERCIO ATACADISTA DE MEDICAMENTOS LTDA',
   'cnpj_do_fabricante': '19.570.700/0000-00',
   'modalidade_da_compra': 'Pregão',
   'descricao_catmat': 'ONDANSETRONA CLORIDRATO, DOSAGEM:2 MG/ML, INDICAÇÃO:INJETÁVEL',
   'anvisa': None,
   'compra': '45294',
   'nome_da_instituicao': 'FUNDO MUNICIPAL DE SAUDE DE SAO JOSE DE CAIANA',
   'cnpj_fornecedor': '31.187.918/0001-15',
   'uf': 'PB'},
  {'insercao': '45477',
   'municipio_da_instituicao': 'DOIS VIZINHOS',
   'quantidade_de_itens_comprados': '1000',
   'preco_unitario': '5,01',
   'codigo_br': '342132',
   'preco_total': '5010',
   'unidade_de_forn